In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd

In [ ]:

# Charger le tokenizer et le modèle BERT pré-entraîné
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Exemple de phrase
k=0
L=[]
file1='/Users/eliotsouthon/Desktop/Projet Dumas/512_sans_lignes_vides.csv'
df = pd.read_csv(file1)
text=[df['Block'][k]]

for k in
# Tokenizer la phrase
inputs = tokenizer(text, return_tensors='pt')

# Récupérer les embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Récupérer les vecteurs de sortie
# `outputs.last_hidden_state` contient les embeddings pour chaque token
# `outputs.pooler_output` contient le vecteur pour le token [CLS]
token_embeddings = outputs.last_hidden_state
cls_embedding = outputs.pooler_output

# Afficher les dimensions des sorties
print("Taille de chaque token embedding :", token_embeddings.size())
print("Taille du CLS embedding :", cls_embedding.size())
print(cls_embedding)


In [ ]:
file1='/Users/eliotsouthon/Desktop/Projet Dumas/fichier_combined.csv'
df = pd.read_csv(file1)

In [ ]:
df.head()

In [ ]:
df['Block'][0]

In [ ]:
text=df['Block'][0]
# Tokenizer la phrase
inputs = tokenizer(text, return_tensors='pt')

# Récupérer les embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Récupérer les vecteurs de sortie
# `outputs.last_hidden_state` contient les embeddings pour chaque token
# `outputs.pooler_output` contient le vecteur pour le token [CLS]
token_embeddings = outputs.last_hidden_state
cls_embedding = outputs.pooler_output

# Afficher les dimensions des sorties
print("Taille de chaque token embedding :", token_embeddings.size())
print("Taille du CLS embedding :", cls_embedding.size())

In [ ]:
pip install datasets

In [7]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification, TFBertModel
from datasets import Dataset

# Lire le fichier CSV
file1 = '/Users/eliotsouthon/Desktop/Projet Dumas/512_sans_lignes_vides.csv'
df = pd.read_csv(file1)

# Diviser les données
train_df, test_df = train_test_split(df, test_size=0.2)

# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Fonction pour tokeniser les données
def tokenize_function(examples):
    return tokenizer(examples['Block'], padding='max_length', truncation=True, max_length=128, return_tensors='tf')

# Convertir les DataFrames en datasets Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokeniser les datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Préparer les datasets TensorFlow
def convert_to_tf_dataset(dataset):
    features = {x: dataset[x] for x in tokenizer.model_input_names}
    labels = dataset['label']
    return tf.data.Dataset.from_tensor_slices((features, labels)).batch(8)

train_tf_dataset = convert_to_tf_dataset(train_dataset)
test_tf_dataset = convert_to_tf_dataset(test_dataset)

# Charger le modèle pré-entraîné BERT pour la classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Utiliser l'optimiseur Keras hérité
legacy_optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=2e-5)

# Configurer l'entraînement
model.compile(optimizer=legacy_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle et capturer l'historique
history = model.fit(train_tf_dataset, epochs=5, validation_data=test_tf_dataset)

# Ajouter une couche Softmax pour obtenir des probabilités
def add_softmax_layer(model):
    # Obtenir les logits du modèle BERT
    logits = model.output
    # Ajouter une couche Softmax
    softmax_output = tf.keras.layers.Softmax()(logits)
    # Créer un nouveau modèle avec la couche Softmax
    return tf.keras.Model(inputs=model.input, outputs=softmax_output)

# Créer le modèle avec la couche Softmax
model_with_softmax = add_softmax_layer(model)

# Exemple d'inférence
def predict_probabilities(texts):
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='tf')
    logits = model_with_softmax(encodings['input_ids'], attention_mask=encodings['attention_mask'])
    return logits.numpy()

# Tester avec un exemple
sample_texts = ["Alors, au fond de ce cœur malade naquit le premier germe d’un ulcère mortel. Cet homme qu’il sacrifiait à son ambition, cet innocent qui ôtait pour son père coupable, lui apparut pâle et menaçant, donnat la main à sa fiancée, pâle comme lui, et trainant après lui le remords non pas celui qui fait bondir le malade comme les furieux de la fatalité antique, mais ce tintement sourd et douloureux qui, à certains moments,frappe sur le cœur et le meurtrit au souvenir d’une action passée, meurtrissure dont les lancinants douleurs creusent un mal qui  va s’approfondissant jusqu’à la mort. ", "Il ne chercha pas à essuyer les larmes qui roulaient sur les joues de Mercedes ; et cependant, pour chacune de ces larmes, il eut donné un verre de son sang ; mais ces larmes coulaient pour un autre."]
probabilities = predict_probabilities(sample_texts)

print("Probabilités pour chaque exemple:")
for text, prob in zip(sample_texts, probabilities):
    print(f"Texte : {text}")
    print(f"Probabilités : {prob}")

# Tracer les courbes de perte et d'exactitude
plt.figure(figsize=(12, 5))

# Courbe de perte
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss (Train)')
plt.plot(history.history['val_loss'], label='Loss (Validation)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over epochs')
plt.legend()

# Courbe d'exactitude
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy (Train)')
plt.plot(history.history['val_accuracy'], label='Accuracy (Validation)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy over epochs')
plt.legend()

plt.tight_layout()
plt.show()


Map: 100%|██████████| 130/130 [00:01<00:00, 117.44 examples/s]
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
65/65 [==============================] - 373s 6s/step - loss: 0.8598 - accuracy: 0.4952 - val_loss: 0.6931 - val_accuracy: 0.4385
Epoch 2/5
65/65 [==============================] - 376s 6s/step - loss: 0.6931 - accuracy: 0.4313 - val_loss: 0.6931 - val_accuracy: 0.4385
Epoch 3/5
65/65 [==============================] - 370s 6s/step - loss: 0.6931 - accuracy: 0.4275 - val_loss: 0.6931 - val_accuracy: 0.4385
Epoch 4/5
65/65 [==============================] - 366s 6s/step - loss: 0.6931 - accuracy: 0.4352 - val_loss: 0.6931 - val_accuracy: 0.4385
Epoch 5/5
65/65 [==============================] - 373s 6s/step - loss: 0.6931 - accuracy: 0.4352 - val_loss: 0.6931 - val_accuracy: 0.4385


AttributeError: Layer tf_bert_for_sequence_classification_2 has no inbound nodes.

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from datasets import Dataset

# Lire le fichier CSV
file1 = '/Users/eliotsouthon/Desktop/Projet Dumas/512_sans_lignes_vides.csv'
df = pd.read_csv(file1)

# Diviser les données
train_df, test_df = train_test_split(df, test_size=0.2)

# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Fonction pour tokeniser les données
def tokenize_function(examples):
    return tokenizer(examples['Block'], padding='max_length', truncation=True, max_length=512, return_tensors='tf')

# Convertir les DataFrames en datasets Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokeniser les datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Préparer les datasets TensorFlow
def convert_to_tf_dataset(dataset):
    features = {x: dataset[x] for x in tokenizer.model_input_names}
    labels = dataset['label']
    return tf.data.Dataset.from_tensor_slices((features, labels)).batch(8)

train_tf_dataset = convert_to_tf_dataset(train_dataset)
test_tf_dataset = convert_to_tf_dataset(test_dataset)

# Charger le modèle pré-entraîné BERT pour la classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Configurer l'entraînement
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Entraîner le modèle
model.fit(train_tf_dataset, epochs=5, validation_data=test_tf_dataset)
